# Create table for companies

In [ ]:
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine


conn_str = "postgresql://localhost:5432/graphyfinancials"
engine = create_engine(conn_str)
connection = engine.connect()

query = """
with companies as (
    select *, row_number() over (partition by company order by cik desc) as row_num
    from dim.companies c 
)
select 
company 
, max(legal_status_form) as legal_status_form 
, max(date_incorporation) as date_incorporation
, max(street1) as street1 
, max(street2) as street2 
, max(city) as city 
, max(state_or_country) as state_or_country 
, max(issuer_website) as issuer_website 
, max(cik) as cik
from companies
where 1=1
and row_num = 1
group by 1
"""

result = connection.execute(query)

df = pd.read_sql(query, engine)
unique_company_name = pd.DataFrame(df['company'].drop_duplicates())

merged_df = pd.merge(unique_company_name,df,on='company')
merged_df